* Meta Llama 3.1 70B Instruct Neuron
*  Mistral 7b
*  BGE large v1.5

In [1]:
%pip install --quiet --upgrade sagemaker jmespath

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-multimodal 1.1.1 requires nvidia-ml-py3==7.352.0, which is not installed.
aiobotocore 2.13.2 requires botocore<1.34.132,>=1.34.70, but you have botocore 1.35.49 which is incompatible.
amazon-sagemaker-sql-magic 0.1.3 requires sqlparse==0.5.0, but you have sqlparse 0.5.1 which is incompatible.
autogluon-core 1.1.1 requires scikit-learn<1.4.1,>=1.3.0, but you have scikit-learn 1.4.2 which is incompatible.
autogluon-features 1.1.1 requires scikit-learn<1.4.1,>=1.3.0, but you have scikit-learn 1.4.2 which is incompatible.
autogluon-multimodal 1.1.1 requires omegaconf<2.3.0,>=2.1.1, but you have omegaconf 2.3.0 which is incompatible.
autogluon-multimodal 1.1.1 requires scikit-learn<1.4.1,>=1.3.0, but you have scikit-learn 1.4.2 which is incompatible.
autogluon-tabular 1.1.1 requires scikit-learn<1.4.1,>=1.3.0

In [2]:
from ipywidgets import Dropdown
from sagemaker.jumpstart.notebook_utils import list_jumpstart_models


dropdown = Dropdown(
    options=list_jumpstart_models("search_keywords includes Text Generation"),
    value="huggingface-llm-mistral-7b-instruct",
    description="Select a JumpStart text generation model:",
    style={"description_width": "initial"},
    layout={"width": "max-content"},
)
display(dropdown)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


Dropdown(description='Select a JumpStart text generation model:', index=48, layout=Layout(width='max-content')…

In [3]:
(
    model_id,
    model_version,
) = (
    "meta-textgeneration-llama-2-7b",
    "2.1.8",
)

In [4]:
from sagemaker.jumpstart.model import JumpStartModel

model = JumpStartModel(model_id=model_id, model_version=model_version)

For forward compatibility, pin to model_version='2.*' in your JumpStartModel or JumpStartEstimator definitions. Note that major version upgrades may have different EULA acceptance terms and input/output signatures.
Using vulnerable JumpStart model 'meta-textgeneration-llama-2-7b' and version '2.1.8'.


In [5]:
# Generate predictor - do not run predictor later in the code
predictor = model.deploy()

-------------------!

In [6]:
def print_response(payload, response):
    print(payload["inputs"])
    print(f"> {response[0]['generation']}")
    print("\n==================================\n")

In [7]:
%%time
payload = {
    "inputs": "I believe the meaning of life is",
    "parameters": {
        "max_new_tokens": 64,
        "top_p": 0.9,
        "temperature": 0.6,
        "return_full_text": False,
    },
}
try:
    response = predictor.predict(payload, custom_attributes="accept_eula=true")  # Changed to true
    print_response(payload, response)
except Exception as e:
    print(e)

I believe the meaning of life is
>  to be happy. We all have different ways of being happy. Some people are happy to have a lot of money and a big house. Some people are happy to have a family and to have their children grow up. And some people are happy to have a career and be successful in their field. I think the most


CPU times: user 11.6 ms, sys: 1.41 ms, total: 13 ms
Wall time: 3.62 s


## Recipes

In [8]:
import json
import sagemaker

from langchain_core.prompts import PromptTemplate
from langchain_community.llms import SagemakerEndpoint
from langchain_community.embeddings import SagemakerEndpointEmbeddings
from langchain_community.llms.sagemaker_endpoint import LLMContentHandler
from langchain_community.embeddings.sagemaker_endpoint import EmbeddingsContentHandler

In [9]:
import json
from typing import List, Dict
from langchain_core.prompts import PromptTemplate
from langchain_community.llms import SagemakerEndpoint
from langchain_community.embeddings import SagemakerEndpointEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
import os

# Additional necessary imports
import pandas as pd  # for CSV and data manipulation
import numpy as np   # for numerical operations
from langchain.document_loaders import CSVLoader  # for loading CSV files
from langchain.document_loaders import DataFrameLoader  # alternative for loading pandas DataFrames

# Optional but commonly needed imports
import boto3  # for AWS services
from tqdm import tqdm  # for progress bars
import matplotlib.pyplot as plt  # for visualizations
#import seaborn as sns  # for enhanced visualizations

# For error handling and debugging
import logging
from typing import Optional, Union  # additional type hints

# If you need to handle different file encodings
import chardet  # for detecting file encodings

# Use the existing endpoint we created
#endpoint_name = predictor.endpoint_name  # Get the endpoint name from your existing predictor

In [10]:
# Deploy the Mistral model (which you already have)
#model_id = "huggingface-llm-mistral-7b-"
#accept_eula = True
#model = JumpStartModel(model_id=model_id, model_version="3.9.0")
#predictor = model.deploy(accept_eula=accept_eula, instance_type="ml.g5.2xlarge")

# Deploy the embedding model
model_id_embedding = "huggingface-sentencesimilarity-bge-large-en-v1-5"
text_embedding_model = JumpStartModel(model_id=model_id_embedding, model_version="1.1.1")
embedding_predictor = text_embedding_model.deploy(accept_eula=True, instance_type="ml.g5.2xlarge")

# Print both endpoint names
print("LLM Endpoint:", predictor.endpoint_name)
print("Embedding Endpoint:", embedding_predictor.endpoint_name)

----------!LLM Endpoint: meta-textgeneration-llama-2-7b-2024-10-27-04-58-15-198
Embedding Endpoint: hf-sentencesimilarity-bge-large-en-v1-5-2024-10-27-05-08-48-999


In [11]:
# Set up constants for endpoints
LLM_ENDPOINT = "meta-textgeneration-llama-2-7b-2024-10-27-04-58-15-198"
EMBEDDING_ENDPOINT = "hf-sentencesimilarity-bge-large-en-v1-5-2024-10-27-05-08-48-999"

In [12]:
class Llama2ContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs: dict) -> bytes:
        payload = {
            "inputs": prompt,
            "parameters": {
                "max_new_tokens": 700,
                "top_p": 0.9,
                "temperature": 0.4,
                "return_full_text": False,  # Important for Llama 2
            },
        }
        input_str = json.dumps(payload)
        return input_str.encode("utf-8")

    def transform_output(self, output: bytes) -> str:
        response_json = json.loads(output.read().decode("utf-8"))
        return response_json[0]["generated_text"].strip()

In [13]:
from typing import List
import json

# Define the embedding
class BGEContentHandlerV15(EmbeddingsContentHandler):
    content_type = "application/json"
    accepts = "application/json"
    
    def transform_input(self, text_inputs: List[str], model_kwargs: dict) -> bytes:
        input_str = json.dumps({
            "text_inputs": text_inputs,
            "mode": "embedding"
        })
        return input_str.encode("utf-8")
    
    def transform_output(self, output: bytes) -> List[List[float]]:
        response_json = json.loads(output.read().decode("utf-8"))
        if isinstance(response_json, list):
            return response_json
        if "embedding" in response_json:
            return response_json["embedding"]
        if "embeddings" in response_json:
            return response_json["embeddings"]
        return response_json[0]

In [14]:
from typing import List
import json
from langchain.embeddings.sagemaker_endpoint import EmbeddingsContentHandler

# Define the embedding
class BGEContentHandlerV15(EmbeddingsContentHandler):
    content_type = "application/json"
    accepts = "application/json"
    
    def transform_input(self, text_inputs: List[str], model_kwargs: dict) -> bytes:
        input_str = json.dumps({
            "text_inputs": text_inputs,
            "mode": "embedding"
        })
        return input_str.encode("utf-8")
    
    def transform_output(self, output: bytes) -> List[List[float]]:
        response_json = json.loads(output.read().decode("utf-8"))
        if isinstance(response_json, list):
            return response_json
        if "embedding" in response_json:
            return response_json["embedding"]
        if "embeddings" in response_json:
            return response_json["embeddings"]
        return response_json[0]

# Then you can create the handler and embeddings
content_handler = BGEContentHandlerV15()
embeddings = SagemakerEndpointEmbeddings(
    endpoint_name=EMBEDDING_ENDPOINT,
    region_name="us-east-1",  # adjust to your region
    content_handler=content_handler
)

In [15]:
# Initialize SageMaker and Endpoints
sess = sagemaker.Session()
region = sess.boto_session.region_name

llm = SagemakerEndpoint(
    endpoint_name="meta-textgeneration-llama-2-7b-2024-10-27-04-58-15-198",
    region_name=region,
    content_handler=Llama2ContentHandler(),
    model_kwargs={"accept_eula": True}  # Changed from custom_attributes
)

sagemaker_embeddings = SagemakerEndpointEmbeddings(
    endpoint_name="hf-sentencesimilarity-bge-large-en-v1-5-2024-10-27-05-08-48-999",
    region_name=region,
    content_handler=BGEContentHandlerV15()
)

In [16]:
!conda install -c pytorch faiss-gpu -y

Channels:
 - pytorch
 - conda-forge
Platform: linux-64
Solving environment: done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - faiss-gpu


The following NEW packages will be INSTALLED:

  faiss-gpu          pytorch/linux-64::faiss-gpu-1.9.0-py3.11_ha2e4562_0_cuda12.1.1 
  libfaiss           pytorch/linux-64::libfaiss-1.9.0-h4818125_0_cuda12.1.1 

The following packages will be UPDATED:

  ca-certificates                       2024.7.4-hbcca054_0 --> 2024.8.30-hbcca054_0 
  certifi                             2024.7.4-pyhd8ed1ab_0 --> 2024.8.30-pyhd8ed1ab_0 
  conda                              24.7.1-py311h38be061_0 --> 24.9.2-py311h38be061_0 
  openssl                                  3.3.1-h4bc722e_2 --> 3.3.1-hb9d3cd8_3 




Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [17]:
import json
from typing import List, Dict
import re  # Added this
import pandas as pd
from langchain_core.prompts import PromptTemplate
from langchain_community.llms import SagemakerEndpoint
from langchain_community.embeddings import SagemakerEndpointEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
import os

In [25]:
# Add this before creating the vector store
df = pd.read_csv("new_recipe_part1.csv")
print("CSV columns:", df.columns.tolist())
print("\nFirst row of data:")
print(df.iloc[0])

CSV columns: ['title_cleaned', 'recipe_new', 'allergy_type', 'diet_type', 'holiday', 'cuisine_type', 'meal_type', 'ingredients_alternatives']

First row of data:
title_cleaned                             Roasted Vegetables with Pecan Pesto
recipe_new                  Roasted Vegetables with Pecan Pesto\n1 1/2 pou...
allergy_type                                       [Nut, Beet, Fruit, Cheese]
diet_type                   [(Vegetarian,80),(Gluten-Free,70),(Dairy-Free,...
holiday                                                          Thanksgiving
cuisine_type                                                American, Italian
meal_type                                                   Dinner, Side dish
ingredients_alternatives    Carrots: sweet potatoes, turnips, rutabaga, ra...
Name: 0, dtype: object


In [36]:
def initialize_recipe_system(csv_path: str) -> FAISS:
    """
    Initialize the recipe system by loading CSV data and creating a FAISS vector store.
    
    Args:
        csv_path (str): Path to the CSV file containing recipes
        
    Returns:
        FAISS: Initialized FAISS vector store with recipe embeddings
    """
    try:
        # Load the CSV file into a pandas DataFrame
        df = pd.read_csv(csv_path)
        print(f"Loaded {len(df)} recipes from CSV")
        
        # Create documents from the DataFrame
        documents = []
        for _, row in df.iterrows():
            # Combine relevant columns into a structured text
            text = f"Title: {row['title_cleaned']}\n"
            text += f"Recipe: {row['recipe_new']}\n"
            text += f"Allergens: {row['allergy_type']}\n"
            text += f"Diet Types: {row['diet_type']}\n"
            text += f"Cuisine: {row['cuisine_type']}\n"
            text += f"Meal Type: {row['meal_type']}\n"
            text += f"Alternative Ingredients: {row['ingredients_alternatives']}\n"
            
            # Create a Document object
            doc = Document(page_content=text)
            documents.append(doc)
        
        print(f"Created {len(documents)} documents")
        
        # Split text into chunks if needed
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1024,
            chunk_overlap=102,
            length_function=len
        )
        split_docs = text_splitter.split_documents(documents)
        print(f"Split into {len(split_docs)} chunks")
        
        # Create and return the FAISS vector store
        vectorstore = FAISS.from_documents(
            documents=split_docs,
            embedding=sagemaker_embeddings
        )
        
        # Verify the vector store was created successfully
        test_query = "test query"
        test_results = vectorstore.similarity_search(test_query, k=1)
        if len(test_results) > 0:
            print("Vector store successfully created and searchable")
            print("\nSample document content:")
            print(test_results[0].page_content[:200], "...")
        
        return vectorstore
    
    except Exception as e:
        raise Exception(f"Failed to initialize recipe system: {str(e)}")

In [37]:
# First, define the content handler with EULA acceptance
from langchain_community.llms.sagemaker_endpoint import LLMContentHandler

class Llama2ContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"
    
    def transform_input(self, prompt: str, model_kwargs: dict) -> bytes:
        payload = {
            "inputs": prompt,
            "parameters": {
                "max_new_tokens": 700,
                "top_p": 0.9,
                "temperature": 0.4,
                "return_full_text": False
            }
        }
        input_str = json.dumps(payload)
        return input_str.encode('utf-8')
    
    def transform_output(self, output: bytes) -> str:
        try:
            response_json = json.loads(output.read().decode("utf-8"))
            print("Raw response:", response_json)
            
            # Handle different response formats
            if isinstance(response_json, list) and len(response_json) > 0:
                if isinstance(response_json[0], dict):
                    # Check for 'generation' key first, then 'generated_text'
                    if 'generation' in response_json[0]:
                        return response_json[0]['generation'].strip()
                    return response_json[0].get("generated_text", "").strip()
                else:
                    return str(response_json[0]).strip()
            elif isinstance(response_json, dict):
                if 'generation' in response_json:
                    return response_json['generation'].strip()
                return response_json.get("generated_text", "").strip()
            else:
                return str(response_json).strip()
                
        except Exception as e:
            print(f"Error parsing model output: {e}")
            print(f"Raw output: {output.read().decode('utf-8')}")
            raise Exception(f"Failed to parse model output: {e}")

# Initialize the SageMaker endpoint
llm = SagemakerEndpoint(
    endpoint_name="meta-textgeneration-llama-2-7b-2024-10-27-04-58-15-198",
    region_name=region,
    content_handler=Llama2ContentHandler(),
    endpoint_kwargs={"CustomAttributes": "accept_eula=true"}  # Changed to endpoint_kwargs
)

# The rest of your code can remain the same
def query_recipes(query: str, vectorstore, allergens: Dict[str, List[str]]):
    try:
        mentioned_allergens = extract_allergens(query, allergens)
        print(f"Allergens found in query: {mentioned_allergens}")
        
        # Get relevant recipes
        docs = vectorstore.similarity_search(query, k=20)
        print(f"Found {len(docs)} relevant recipes")
        
        if mentioned_allergens:
            filtered_docs = []
            for doc in docs:
                content_lower = doc.page_content.lower()
                if not any(allergen.lower() in content_lower for allergen in mentioned_allergens):
                    filtered_docs.append(doc)
            docs = filtered_docs[:10]
            print(f"Filtered to {len(docs)} documents after removing allergen-containing content")
        
        # Create a more structured prompt
        prompt = f"""[INST] You are a helpful cooking assistant. Create a recipe based on this request:

REQUEST: {query}
ALLERGIES TO AVOID: {', '.join(mentioned_allergens) if mentioned_allergens else 'None'}

Here are some relevant recipes for reference:
{docs[0].page_content if docs else 'No reference recipes available'}

Please provide:
1. List of ingredients with measurements
2. Step-by-step instructions
3. Any tips or variations

Format the recipe clearly with sections for ingredients and instructions.
[/INST]"""
        
        print("\nGenerating recipe...")
        try:
            response = llm.invoke(prompt)
            print("\nResponse received, length:", len(response) if response else 0)
            
            if not response or len(response) < 50:
                print("Response is too short, retrying...")
                retry_prompt = f"""[INST] Create a simple recipe for {query}. Include:
1. Ingredients list with measurements
2. Step-by-step cooking instructions
[/INST]"""
                response = llm.invoke(retry_prompt)
            
            return response if response else "Error: Empty response from model"
            
        except Exception as e:
            print(f"Error during recipe generation: {str(e)}")
            return f"Error generating recipe: {str(e)}"
            
    except Exception as e:
        print(f"Error in query_recipes: {str(e)}")
        return f"Error processing recipe query: {str(e)}"
        
# Also add the extract_allergens function
def extract_allergens(query: str, allergens: Dict[str, List[str]]) -> List[str]:
    """Extract mentioned allergens from the query."""
    mentioned_allergens = []
    query_lower = query.lower()
    
    for category, allergen_list in allergens.items():
        for allergen in allergen_list:
            if allergen.lower() in query_lower:
                mentioned_allergens.append(allergen)
    
    return mentioned_allergens

# Then your initialization code remains the same
try:
    csv_path = "new_recipe_part1.csv"
    vectorstore = initialize_recipe_system(csv_path)
    print("Vector store created successfully!")
    
    allergens_dict = {
        "common": ["milk", "eggs", "peanuts", "tree nuts", "soy", "wheat", "fish", "shellfish"],
        "additional": ["sesame", "gluten", "dairy"]
    }

    result = query_recipes("Make me a chocolate cake", vectorstore, allergens_dict)
    print(result)
    
except Exception as e:
    print(f"Error during initialization: {str(e)}")

Loaded 7446 recipes from CSV
Created 7446 documents
Split into 21529 chunks
Vector store successfully created and searchable

Sample document content:
Instructions: ...
Vector store created successfully!
Allergens found in query: []
Found 20 relevant recipes

Generating recipe...
Raw response: [{'generation': '\n\n### [INST] You are a helpful cooking assistant. Create a recipe based on this request:\n\nREQUEST: Make me a chocolate cake\nALLERGIES TO AVOID: None\n\nHere are some relevant recipes for reference:\nTitle: Chocolate Cake\nRecipe: Chocolate Cake\n2 cups sugar\n1 cup unsweetened cocoa\n1 cup vegetable shortening\n1 teaspoon salt\n2 teaspoons baking powder\n1 teaspoon baking soda\n3 cups all-purpose flour\n2 eggs, at room temperature\n1 cup milk, at room temperature\n1 teaspoon almond extract\n1 teaspoon vanilla extract\n1 cup hot, strong coffee\n\nPlease provide:\n1. List of ingredients with measurements\n2. Step-by-step instructions\n3. Any tips or variations\n\nFormat the re

In [38]:
# allergy = Nut & Treenut / cousine = Thai
# Example Usage
query = "I am allergic to nuts and tree nuts. Can you suggest a Thai curry recipe that is completely nut-free?"
response = query_recipes(query, vectorstore, allergens_dict)
print(response)

Allergens found in query: ['tree nuts']
Found 20 relevant recipes
Filtered to 10 documents after removing allergen-containing content

Generating recipe...
Raw response: [{'generation': "\n\n### 3.2.1\n\n[INST] You are a helpful cooking assistant. Create a recipe based on this request:\n\nREQUEST: I am allergic to nuts and tree nuts. Can you suggest a Thai curry recipe that is completely nut-free?\nALLERGIES TO AVOID: tree nuts\n\nHere are some relevant recipes for reference:\nAllergens: [Peanut, Beef, Ginger, Cumin, Basil, Fish]\nDiet Types: [(Vegetarian,80),(Gluten-Free,70),(Dairy-Free,60),(Lactovegetarians,50),(Paleo,40)]\nCuisine: Thai\nMeal Type: Dinner, Lunch, Appetizer\nAlternative Ingredients: Mung bean sprouts: broccoli sprouts, bok choy, cabbage\nKohlrabi: turnip, radish, rutabaga\nCarrots: beets, parsnips, sweet potatoes\nSpinach: kale, collard greens, chard\nCherry tomatoes: grape tomatoes, plum tomatoes, apricot tomatoes\nTamarind paste: date paste, fig paste, prune paste\

In [39]:
# allergy = Gluten (Wheat) / cousine = Italian
# Example Usage
query = "I have celiac disease. Please suggest an Italian polenta-based recipe that's naturally gluten-free."
response = query_recipes(query, vectorstore, allergens_dict)
print(response)

Allergens found in query: ['gluten']
Found 20 relevant recipes
Filtered to 10 documents after removing allergen-containing content

Generating recipe...
Raw response: [{'generation': "\n\n### [INST] You are a helpful cooking assistant. Create a recipe based on this request:\n\nREQUEST: I have celiac disease. Please suggest an Italian polenta-based recipe that's naturally gluten-free.\nALLERGIES TO AVOID: gluten\n\nHere are some relevant recipes for reference:\nTitle: Sweet Polenta Crostini with Mascarpone, Raspberries, Pistachios, and Lavender Honey\nRecipe: Sweet Polenta Crostini with Mascarpone, Raspberries, Pistachios, and Lavender Honey\n3 1/2 cups water, preferably still spring or filtered water\n1/2 teaspoon salt\n1 cup medium grain yellow cornmeal, preferably stone-ground\n4 tablespoons unsalted butter\n1 tablespoon mild-flavored vegetable oil, plus extra for greasing the pan\n2 to 3 tablespoons granulated sugar\n3/4 cup mascarpone\n1/2 pint red raspberries\n1/3 cup shelled, ski

In [40]:
# allergy = Milk & Egg / cousine = Dessert
# Example Usage
query = "Looking for an ice cream alternative - I'm allergic to milk and eggs. Can you suggest a frozen dessert recipe?"
response = query_recipes(query, vectorstore, allergens_dict)
print(response)

Allergens found in query: ['milk', 'eggs']
Found 20 relevant recipes
Filtered to 1 documents after removing allergen-containing content

Generating recipe...
Raw response: [{'generation': '\n\n### 5. Make a [Vegan] [Dairy-Free] [Gluten-Free] [Paleo] [Pescatarian] [Fruitarian] [Vegetarian] [Vegetarian] [Vegetarian] [Vegetarian] [Vegetarian] [Vegetarian] [Vegetarian] [Vegetarian] [Vegetarian] [Vegetarian] [Vegetarian] [Vegetarian] [Vegetarian] [Vegetarian] [Vegetarian] [Vegetarian] [Vegetarian] [Vegetarian] [Vegetarian] [Vegetarian] [Vegetarian] [Vegetarian] [Vegetarian] [Vegetarian] [Vegetarian] [Vegetarian] [Vegetarian] [Vegetarian] [Vegetarian] [Vegetarian] [Vegetarian] [Vegetarian] [Vegetarian] [Vegetarian] [Vegetarian] [Vegetarian] [Vegetarian] [Vegetarian] [Vegetarian] [Vegetarian] [Vegetarian] [Vegetarian] [Vegetarian] [Vegetarian] [Vegetarian] [Vegetarian] [Vegetarian] [Vegetarian] [Vegetarian] [Vegetarian] [Vegetarian] [Vegetarian] [Vegetarian] [Vegetarian] [Vegetarian] [Vegetar

In [41]:
# allergy = Soy / cousine = Vegan
# Example Usage
query = "I am allergic to soy and eat vegan. Can you share a recipe for a protein-rich main dish using legumes instead of soy products?"
response = query_recipes(query, vectorstore, allergens_dict)
print(response)

Allergens found in query: ['soy']
Found 20 relevant recipes
Filtered to 5 documents after removing allergen-containing content

Generating recipe...
Raw response: [{'generation': "\n\n### 2. [INST] You are a helpful cooking assistant. Create a recipe based on this request:\n\nREQUEST: I am allergic to soy and eat vegan. Can you share a recipe for a protein-rich main dish using legumes instead of soy products?\nALLERGIES TO AVOID: soy\n\nHere are some relevant recipes for reference:\n16. Serve tofu with snap pea salad and apricot dipping sauce.\nAllergens: [Cow's Milk, Eggs, Coconut]\nDiet Types: [(Vegan,90),(Vegetarian,90),(Gluten-Free,80),(Dairy-Free,70),(Fruitarian,60),(Paleo,50)]\nCuisine: American, Asian\nMeal Type: Dinner, Appetizer\nAlternative Ingredients: Tofu: tempeh, seitan, vegan sausage\nKosher salt: sea salt\nFlour: all-purpose flour, whole wheat flour\nEggs: flaxseed, chia seeds, aquafaba\nRed pepper flakes: cayenne pepper, paprika\nCoconut: cashews, macadamia nuts\nBread

In [42]:
# allergy = Fish & Crustacean / cousine = Japanese
# Example Usage
query = "I have seafood allergies (fish and shellfish). Can you provide a recipe for vegetable tempura with a dipping sauce that doesn't contain dashi or fish sauce?"
response = query_recipes(query, vectorstore, allergens_dict)
print(response)

Allergens found in query: ['fish', 'shellfish']
Found 20 relevant recipes
Filtered to 7 documents after removing allergen-containing content

Generating recipe...
Raw response: [{'generation': "\n\n### [INST] You are a helpful cooking assistant. Create a recipe based on this request:\n\nREQUEST: I have a nut allergy. Can you provide a recipe for a dessert that doesn't contain nuts?\nALLERGIES TO AVOID: nuts\n\nHere are some relevant recipes for reference:\nTitle: Vegan Chocolate Mousse\nRecipe: Vegan Chocolate Mousse\n1/2 cup unsweetened cocoa powder\n1/2 cup soy milk\n1/2 cup coconut milk\n1/2 cup maple syrup\n1/4 cup agave syrup\n1/4 cup peanut butter\n1/2 teaspoon vanilla extract\n1/2 teaspoon salt\n1/2 teaspoon baking soda\n1/2 teaspoon xanthan gum\n\nInstructions:\n1. In a food processor, combine cocoa powder, soy milk, coconut milk, maple syrup, agave syrup, peanut butter, vanilla extract, salt, and baking soda.\n2. Process until smooth.\n3. Pour into a bowl and refrigerate for 1